In [ ]:
import numpy as np

data = np.load(r"data\ssdsdsd\平静\Logitech_cam\1755948633.748267f30c1550.npz")
frames = data['frames']
frame_lens = data['frame_lens']


In [ ]:
import cv2
for i,frame in enumerate(frames):
    frame_data = frame[:frame_lens[i]]
    # cv2解码
    img = cv2.imdecode(frame_data, cv2.IMREAD_COLOR_BGR)
    cv2.imshow('img',img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
from PIL import Image
import numpy as np
import cv2
import av
import io

# 读取文件
data = np.load(r"D:\work\主动健康机器人\orbbec\data\99999.npz")
frames = data['frames']
frame_lens = data['frame_lens']



In [ ]:
for frame in frames:
    print(len(frame))

In [ ]:
bty = frames[0][:frame_lens[0]]
buf = io.BytesIO()
print(len(frames[3]))
print(frame_lens[3])
container = av.open(buf, format='h264')
stream = container.streams.video[0]
container.close()
for i, frame_data in enumerate(frames):
    data = frame_data[:frame_lens[i]]
    tmp = av.packet.Packet(data)
    tmp.stream = stream
    img = tmp.decode()[0]
    img = img.to_ndarray(format='bgr24')
    cv2.imshow('frame',img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
import subprocess
import threading
import queue
import time

def reader_thread(pipe, q):
    while True:
        data = pipe.read(4096)
        if not data:
            break
        q.put(data)
    pipe.close()

process = subprocess.Popen([
    'ffmpeg',
    '-f', 'dshow',
    '-video_size', '1920x1080',
    '-framerate', '30',
    '-vcodec', 'mjpeg',
    '-i', 'video=HD Pro Webcam C920',
    '-f', 'mjpeg',
    '-c:v', 'copy',
    '-'
], stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=10**8)

q = queue.Queue()
t = threading.Thread(target=reader_thread, args=(process.stdout, q))
t.daemon = True
t.start()

buffer = b''
frame_count = 0
bytes_list = []

start_time = time.time()
while True:
    try:
        chunk = q.get(timeout=1)  # 等待1秒数据
    except queue.Empty:
        # 超时无数据，判断是否退出
        if time.time() - start_time > 10:
            break
        else:
            continue

    buffer += chunk
    print(f"读取到 {len(buffer)} 字节的数据。")
    while True:
        start_marker = buffer.find(b'\xff\xd8')
        end_marker = buffer.find(b'\xff\xd9')
        if start_marker != -1 and end_marker != -1 and start_marker < end_marker:
            frame_data = buffer[start_marker:end_marker + 2]
            buffer = buffer[end_marker + 2:]
            bytes_list.append(frame_data)
        else:
            break

process.terminate()
t.join()

print(f"总共捕获了 {len(bytes_list)} 帧。")


读取到 4096 字节的数据。
读取到 8192 字节的数据。
读取到 12288 字节的数据。
读取到 16384 字节的数据。
读取到 20480 字节的数据。
读取到 24576 字节的数据。
读取到 28672 字节的数据。
读取到 32768 字节的数据。
读取到 36864 字节的数据。
读取到 40960 字节的数据。
读取到 45056 字节的数据。
读取到 49152 字节的数据。
读取到 53248 字节的数据。
读取到 57344 字节的数据。
读取到 61440 字节的数据。
读取到 65536 字节的数据。
读取到 69632 字节的数据。
读取到 73728 字节的数据。
读取到 77824 字节的数据。
读取到 81920 字节的数据。
读取到 86016 字节的数据。
读取到 90112 字节的数据。
读取到 94208 字节的数据。
读取到 98304 字节的数据。
读取到 102400 字节的数据。
读取到 106496 字节的数据。
读取到 110592 字节的数据。
读取到 114688 字节的数据。
读取到 118784 字节的数据。
读取到 122880 字节的数据。
读取到 126976 字节的数据。
读取到 131072 字节的数据。
读取到 135168 字节的数据。
读取到 139264 字节的数据。
读取到 143360 字节的数据。
读取到 147456 字节的数据。
读取到 151552 字节的数据。
读取到 155648 字节的数据。
读取到 159744 字节的数据。
读取到 163840 字节的数据。
读取到 167936 字节的数据。
读取到 172032 字节的数据。
读取到 176128 字节的数据。
读取到 180224 字节的数据。
读取到 184320 字节的数据。
读取到 188416 字节的数据。
读取到 192512 字节的数据。
读取到 196608 字节的数据。
读取到 200704 字节的数据。
读取到 204800 字节的数据。
读取到 208896 字节的数据。
读取到 212992 字节的数据。
读取到 217088 字节的数据。
读取到 221184 字节的数据。
读取到 225280 字节的数据。
读取到 5546 字节的数据。
读取到 9642 字节的数据。
读取到 

KeyboardInterrupt: 

In [11]:
from PIL import Image
import numpy as np
import cv2
import av
import io
bty = bytes_list[0]
buf = io.BytesIO()
buf.write(bty)
container = av.open(buf, format='mjpeg')
stream = container.streams.video[0]
container.close()
cont = 0
start = time.time()
for i, data in enumerate(bytes_list):
    tmp = av.packet.Packet(data)
    tmp.stream = stream
    img = tmp.decode()[0]
    img = img.to_ndarray(format='bgr24')
    # cv2.imshow('frame',img)
    # key = cv2.waitKey(1)
    # if key == ord('q'):
    #     break
    cont += 1
    if time.time() - start > 1:
        print(f"fps: {cont}")
        cont = 0
        start = time.time()
        
cv2.destroyAllWindows()

fps: 168


In [17]:
import numpy as np

data = np.load(r"data\测试9\平静\ppg\1756212640.660101f1000c4698.npz",allow_pickle=True)
frames = data['frames']
timestamps = data['timestamps']
meta_info = data['meta_info']

In [ ]:
for i in range(3):
    print(timestamps[1000*(i+1)]-timestamps[1000*i])

1.0024387836456299